# 2024 Day 8: Resonant Collinearity

## Part 1

You find yourselves on the roof of a top-secret Easter Bunny installation.

While The Historians do their thing, you take a look at the familiar huge antenna. Much to your surprise, it seems to have been reconfigured to emit a signal that makes people 0.1% more likely to buy Easter Bunny brand Imitation Mediocre Chocolate as a Christmas gift! Unthinkable!

Scanning across the city, you find that there are actually many such antennas. Each antenna is tuned to a specific frequency indicated by a single lowercase letter, uppercase letter, or digit. You create a map (your puzzle input) of these antennas. For example:

```
............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............
```

The signal only applies its nefarious effect at specific antinodes based on the resonant frequencies of the antennas. In particular, an antinode occurs at any point that is perfectly in line with two antennas of the same frequency - but only when one of the antennas is twice as far away as the other. This means that for any pair of antennas with the same frequency, there are two antinodes, one on either side of them.

So, for these two antennas with frequency a, they create the two antinodes marked with #:

```
..........
...#......
..........
....a.....
..........
.....a....
..........
......#...
..........
..........
```

Adding a third antenna with the same frequency creates several more antinodes. It would ideally add four antinodes, but two are off the right side of the map, so instead it adds only two:

```
..........
...#......
#.........
....a.....
........a.
.....a....
..#.......
......#...
..........
..........
```

Antennas with different frequencies don't create antinodes; A and a count as different frequencies. However, antinodes can occur at locations that contain antennas. In this diagram, the lone antenna with frequency capital A creates no antinodes but has a lowercase-a-frequency antinode at its location:

```
..........
...#......
#.........
....a.....
........a.
.....a....
..#.......
......A...
..........
..........
```

The first example has antennas with two different frequencies, so the antinodes they create look like this, plus an antinode overlapping the topmost A-frequency antenna:

```
......#....#
...#....0...
....#0....#.
..#....0....
....0....#..
.#....A.....
...#........
#......#....
........A...
.........A..
..........#.
..........#.
```

Because the topmost A-frequency antenna overlaps with a 0-frequency antinode, there are 14 total unique locations that contain an antinode within the bounds of the map.

Calculate the impact of the signal. How many unique locations within the bounds of the map contain an antinode?

In [1]:
# Loading the antenna map (puzzle input) from file
with open('aoc-2024-day-08.txt') as f:
    antenna_map = f.read().splitlines()
    
# Setting the sample data
sample_data = '''............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............'''.splitlines()

# Overwriting the input for testing  purposes
# (Note: Comment this line out when teady to use full sample input)
# antenna_map = sample_data

In [2]:
# Instantiating a dictionary to hold the locations of the antennas by frequency
antenna_locations = {}

# Iterating through each line of the antenna map
for line in antenna_map:

    # Iterating over each character in the line
    for i, char in enumerate(line):
        
        # Adding the location of the antenna to the dictionary
        if char != '.':
            if char not in antenna_locations:
                antenna_locations[char] = []
            antenna_locations[char].append((antenna_map.index(line), i))

In [3]:
def determine_antinodes(base_antenna, distant_antenna):
    '''
    Determines the locations of antinodes based on base and distant antennas

    Inputs:
        - base_antenna (tuple): tuple representing the location of the base antenna
        - distant_antenna (tuple): tuple representing the location of the distant antenna

    Returns:
        - antinodes (list): list of tuples representing the locations of the antinodes
    '''

    # Getting the distance changes between the base and distant antenna
    dx = distant_antenna[1] - base_antenna[1]
    dy = distant_antenna[0] - base_antenna[0]

    # Determining the locations of the antinodes
    antinode1 = (base_antenna[0] - dy, base_antenna[1] - dx)
    antinode2 = (distant_antenna[0] + dy, distant_antenna[1] + dx)

    # Determining the appropriate height and width of the antenna map
    height = len(antenna_map)
    width = len(antenna_map[0])

    # Checking if the antinodes are in bounds
    antinode1_in_bounds = 0 <= antinode1[0] < height and 0 <= antinode1[1] < width
    antinode2_in_bounds = 0 <= antinode2[0] < height and 0 <= antinode2[1] < width

    # Appending the antinodes to a list if in bounds
    antinodes = []
    if antinode1_in_bounds:
        antinodes.append(antinode1)
    if antinode2_in_bounds:
        antinodes.append(antinode2)

    return antinodes

In [4]:
from itertools import combinations

# Creating a set to store unique antinode locations
unique_antinodes = set()

# Iterating through the antenna locations
for antenna, freqs in antenna_locations.items():
    
    # Getting all possible pairs of antennas with the same frequency
    for base_antenna, distant_antenna in combinations(freqs, 2):
        
        # Determining the antinodes
        antinodes = determine_antinodes(base_antenna, distant_antenna)
        
        # Adding antinodes to the set (duplicates will be automatically removed)
        unique_antinodes.update(antinodes)

print(f'Total number of unique antinodes: {len(unique_antinodes)}')

Total number of unique antinodes: 285


## Part 2
Watching over your shoulder as you work, one of The Historians asks if you took the effects of resonant harmonics into your calculations.

Whoops!

After updating your model, it turns out that an antinode occurs at any grid position exactly in line with at least two antennas of the same frequency, regardless of distance. This means that some of the new antinodes will occur at the position of each antenna (unless that antenna is the only one of its frequency).

So, these three T-frequency antennas now create many antinodes:

```
T....#....
...T......
.T....#...
.........#
..#.......
..........
...#......
..........
....#.....
..........
```

In fact, the three T-frequency antennas are all exactly in line with two antennas, so they are all also antinodes! This brings the total number of antinodes in the above example to 9.

The original example now has 34 antinodes, including the antinodes that appear on every antenna:

```
##....#....#
.#.#....0...
..#.#0....#.
..##...0....
....0....#..
.#...#A....#
...#..#.....
#....#.#....
..#.....A...
....#....A..
.#........#.
...#......##
```

Calculate the impact of the signal using this updated model. How many unique locations within the bounds of the map contain an antinode?

In [5]:
def determine_additional_antinodes(base_antenna, distant_antenna):
    '''
    Determines the additional locations of antinodes based on base and distant antennas

    Inputs:
        - base_antenna (tuple): tuple representing the location of the base antenna
        - distant_antenna (tuple): tuple representing the location of the distant antenna

    Returns:
        - antinodes (list): list of tuples representing the locations of the antinodes
    '''

    # Getting the distance changes between the base and distant antenna
    dx = distant_antenna[1] - base_antenna[1]
    dy = distant_antenna[0] - base_antenna[0]

    # Determine the GCD of dx and dy to find the smallest step size
    def gcd(a, b):
        a, b = abs(a), abs(b)
        while b:
            a, b = b, a % b
        return a

    # If both dx and dy are 0, there's no line
    if dx == 0 and dy == 0:
        return []

    # Calculate the step size
    step = gcd(dx if dx != 0 else dy, dy if dy != 0 else dx)
    dx_step = dx // step
    dy_step = dy // step

    # Determining the appropriate height and width of the antenna map
    height = len(antenna_map)
    width = len(antenna_map[0])

    # Finding all antinodes by stepping in both directions
    antinodes = []
    curr_pos = base_antenna

    # Steping backwards until we hit the boundary
    while 0 <= curr_pos[0] < height and 0 <= curr_pos[1] < width:
        antinodes.append(curr_pos)
        curr_pos = (curr_pos[0] - dy_step, curr_pos[1] - dx_step)

    # Starting from distant_antenna and step forward
    curr_pos = distant_antenna
    while 0 <= curr_pos[0] < height and 0 <= curr_pos[1] < width:
        antinodes.append(curr_pos)
        curr_pos = (curr_pos[0] + dy_step, curr_pos[1] + dx_step)

    return antinodes

In [6]:
from itertools import combinations

# Creating a set to store unique antinode locations
unique_antinodes = set()

# Iterating through the antenna locations
for antenna, freqs in antenna_locations.items():
    
    # Getting all possible pairs of antennas with the same frequency
    for base_antenna, distant_antenna in combinations(freqs, 2):
        
        # Determining the antinodes
        antinodes = determine_additional_antinodes(base_antenna, distant_antenna)
        
        # Adding antinodes to the set (duplicates will be automatically removed)
        unique_antinodes.update(antinodes)

print(f'Total number of unique antinodes: {len(unique_antinodes)}')

Total number of unique antinodes: 944
